Authors:
    Rodrigo Perez-Dattari, <r.j.perezdattari@tudelft.nl>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

#### Useful functions

In [ ]:
# Plot
def plot_joints_data(data, time, title, subtitle, x_label, y_label):
    # Create subplots
    fig, axs = plt.subplots(n_joints, figsize=(10,15))

    # Add title of subplots 
    fig.suptitle(title)

    # Plot every joint
    for i in range(n_joints):
        axs[i].set_title(subtitle + ' ' + str(i + 1))
        axs[i].set_xlabel(x_label)
        axs[i].set_ylabel(y_label)
        axs[i].scatter(time, data[:, i])

    fig.tight_layout()

In [ ]:
# Find index of closest value in array
def find_index_closest_value(array, value):
    # Calculate the difference array
    difference_array = np.absolute(array - value)

    # Find the index of minimum element from the array
    index = difference_array.argmin()
    
    return index

In [ ]:
# Compute time
def compute_time(delta_t_history):
    time_history = []
    time = 0

    # Sum every delta to get the time of the trajectory
    for delta_t in delta_t_history:
        time = time + delta_t
        time_history.append(time)
        
    return time_history

### 1. Plot Original Data

#### Load demonstration

In [ ]:
filename = 'joints_state_0.pk'
folder = 'demonstrations/'
with open(folder + filename, 'rb') as file:
    demonstration = pickle.load(file)

#### Compute time

In [ ]:
delta_t_history = np.array(demonstration['delta_t'])
time_history = compute_time(delta_t_history)

#### Plot joints positions

In [ ]:
# Get joint positions
q_trajectory = np.array(demonstration['q'])
n_joints = q_trajectory.shape[1]

In [ ]:
plot_joints_data(q_trajectory, time_history, title='Joints Positions', subtitle='Joint', x_label='time [s]', y_label='angle [rad]')

#### Plot joints velocities

In [ ]:
# Get joint velocities
q_dot_trajectory = np.array(demonstration['q_dot'])
n_joints = q_dot_trajectory.shape[1]

In [ ]:
plot_joints_data(q_dot_trajectory, time_history, title='Joints Velocities', subtitle='Joint', x_label='time [s]', y_label='velocity [rad/s]')

### 2. Trim Data and Plot

#### Remove edges demonstrations 

In [ ]:
# Define demonstration start and end time 
start_time = 25
end_time = 175

In [ ]:
# Get indexes
start_index = find_index_closest_value(np.array(time_history), start_time)
end_index = find_index_closest_value(np.array(time_history), end_time)

In [ ]:
# Remove edges
q_trajectory_trimmed = q_trajectory[start_index:end_index]
q_dot_trajectory_trimmed = q_dot_trajectory[start_index:end_index]
delta_t_history_trimmed = delta_t_history[start_index:end_index]
time_history_trimmed = compute_time(delta_t_history_trimmed)

#### Plot new joint positions

In [ ]:
plot_joints_data(q_trajectory_trimmed, time_history_trimmed, title='Joints Positions New', subtitle='Joint', x_label='time [s]', y_label='angle [rad]')

#### Plot new joint velocities

In [ ]:
plot_joints_data(q_dot_trajectory_trimmed, time_history_trimmed, title='Joints Velocities New', subtitle='Joint', x_label='time [s]', y_label='angle [rad]')

### 3. Subsample data and Plot

In [ ]:
subsample_factor = 3

In [ ]:
# Subsample arrays
q_trajectory_subsampled = q_trajectory_trimmed[::subsample_factor]
q_dot_trajectory_subsampled = q_dot_trajectory_trimmed[::subsample_factor]
delta_t_history_subsampled = delta_t_history_trimmed[::subsample_factor] * subsample_factor
time_history_subsampled = time_history_trimmed[::subsample_factor]

In [ ]:
# Compare trajectory lenghts
original_length = q_trajectory.shape[0]
trimmed_length = q_trajectory_trimmed.shape[0]
subsampled_length = q_trajectory_subsampled.shape[0]

print('Original trajectory length:', original_length)
print('Trimmed trajectory length:', trimmed_length)
print('Subsampled trajectory length:', subsampled_length)

In [ ]:
# Check new sample frequency mean
mean_sampling_frequency = 1 / np.array(delta_t_history_subsampled).mean()
print('Sampling frequency:', mean_sampling_frequency)

In [ ]:
# Plot positions
plot_joints_data(q_trajectory_subsampled, time_history_subsampled, title='Joints Positions New', subtitle='Joint', x_label='time [s]', y_label='angle [rad]')

In [ ]:
# Plot velocities
plot_joints_data(q_dot_trajectory_subsampled, time_history_subsampled, title='Joints Positions New', subtitle='Joint', x_label='time [s]', y_label='angle [rad]')

#### Save processed data

In [ ]:
# Create dictionary
trajectory = {'q': q_trajectory_subsampled,
              'q_dot': q_dot_trajectory_subsampled,
              'delta_t': delta_t_history_subsampled,
              'time': time_history_subsampled}

# Save dictionary
folder = 'demonstrations_post/'  # make sure the folder exists!
filename = 'post_' + filename
with open(folder + filename, 'wb') as file:
    pickle.dump(trajectory, file)